In [1]:
import chromadb
import openai
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import datetime as dt

API_KEY = os.getenv('OAI_API_KEY')


In [2]:
df = pd.read_csv('./data/saigai_data.csv')
df = df.astype({'年': 'Int64', '月': 'Int64', '業種（中分類）': 'Int64', '業種（小分類）': 'Int64', '事故の型': 'Int64', '起因物（大分類）.2': 'Int64', '起因物（中分類）.2': 'Int64', '起因物（小分類）.2': 'Int64'})
df_new = df[['年号','年','月','災害状況','業種（大分類）.1']]

/var/folders/5s/lns070lx7x3_4_7qbhgzsj0w0000gn/T/ipykernel_15090/2698770361.py:1: DtypeWarning: Columns (7,14,16,17,18,22,24,26,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/saigai_data.csv')


In [3]:
client = chromadb.Client()

In [4]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=API_KEY,
                model_name="text-embedding-ada-002"
            )
collection = client.create_collection(name="disaster_collection",embedding_function=openai_ef)


In [12]:
df_dic = df_new[:1000].to_dict()
docs = list(df_dic['災害状況'].values())
# df_dicから'災害状況'キーを除いた辞書を取り出すコード
original_data = {key: value for key, value in df_dic.items() if key != '災害状況'}
metadata = [
    {key: value[index] for key, value in original_data.items()}
    for index in range(len(next(iter(original_data.values()))))
]

ids = ["id{}".format(i) for i in df_dic['災害状況'].keys()]



In [13]:
collection.add(
    documents=docs,
    metadatas=metadata,
    ids=ids
)

In [14]:
results = collection.query(
    query_texts=["季節性災害"],
    n_results=10
)
results

{'ids': [['id128',
   'id432',
   'id436',
   'id615',
   'id191',
   'id116',
   'id286',
   'id611',
   'id6',
   'id56']],
 'distances': [[0.3918856382369995,
   0.3959372937679291,
   0.39612558484077454,
   0.3994092345237732,
   0.4008343815803528,
   0.40150177478790283,
   0.40164804458618164,
   0.40415099263191223,
   0.4056999683380127,
   0.4065854847431183]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '保健・衛生業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '保健・衛生業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'}]],
 'embeddings': None,
 'documents': [['コンテナより荷卸しした製材をリフトにより倉庫内に

In [15]:
results = collection.query(
    query_texts=["虫刺され"],
    n_results=10
)
results

{'ids': [['id431',
   'id49',
   'id547',
   'id679',
   'id782',
   'id631',
   'id377',
   'id374',
   'id407',
   'id304']],
 'distances': [[0.2651117742061615,
   0.2969682216644287,
   0.29818081855773926,
   0.2985743582248688,
   0.2991569936275482,
   0.2995659410953522,
   0.3005407154560089,
   0.3006022572517395,
   0.3009767532348633,
   0.3017663359642029]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '接客娯楽業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '接客娯楽業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '清掃・と畜業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '接客娯楽業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'}]],
 'embeddings': None,
 'documents': [['調理場内洗い場にて、洗いものをしていた際、シンクの中

In [17]:
results = collection.query(
    query_texts=["不注意"],
    n_results=5
)
results

{'ids': [['id934', 'id167', 'id53', 'id395', 'id394']],
 'distances': [[0.40046584606170654,
   0.4175884425640106,
   0.4180261492729187,
   0.4215961992740631,
   0.4230976998806]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '建設業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '金融・広告業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '商業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': 'その他の事業'}]],
 'embeddings': None,
 'documents': [['資材整理中、周辺が暗かった為、地面の凸凹に気付かず、躓き、転倒。手首を強打し、負傷した。',
   '自宅訪問中、対向車が車線をはずれ、衝突した。',
   '休憩室にて、作業服に着替える際、誤って転倒。足甲を骨折した。',
   '国道を走行中、右折レーンの最後尾の停止した車両と前方不注意により、追突した。',
   '国道を走行中、右折レーンの最後尾の停止した車両と前方不注意により、追突した。']],
 'uris': None,
 'data': None}

In [18]:
results = collection.query(
    query_texts=["交通事故"],
    n_results=5
)
results

{'ids': [['id333', 'id292', 'id79', 'id939', 'id173']],
 'distances': [[0.2983427941799164,
   0.2989140450954437,
   0.30171361565589905,
   0.3021944463253021,
   0.30408617854118347]],
 'metadatas': [[{'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '製造業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'},
   {'年': 25, '年号': '平成', '月': 1, '業種（大分類）.1': '運輸交通業'}]],
 'embeddings': None,
 'documents': [['製品を客先へ搬送中、交差点にて、赤信号のために停止していたところ、後方から来た車に追突され、衝撃により首を負傷した。',
   '残土搬出後現場に戻った際、車両２台で連なり走行中、前方の車両が路側に停車したところ、後方の車両が追突。前方の車両の運転手は怪我はなかったが、後方の車両を運転していた被災者は、大腿骨を骨折した。',
   '市交差点にて、赤信号停止中、後方から走行して来た車両に追突された。',
   'タクシーで走行中、交差点の信号で停止した際、後方から追突された。',
   '客車の運転代行中、道路の確認の為、停車したところ、後続の随伴車両に追突された。']],
 'uris': None,
 'data': None}